In [0]:
%sql
CREATE TABLE IF NOT EXISTS dbms.gold.dim_patient
SHALLOW CLONE dbms.silver.patient;


source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
31876,1,null,0,0,0,0


In [0]:
%sql
CREATE TABLE IF NOT EXISTS dbms.gold.dim_study
SHALLOW CLONE dbms.silver.study

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
16255,1,null,0,0,0,0


In [0]:
%sql
CREATE TABLE IF NOT EXISTS dbms.gold.dim_series AS
SELECT
  s.*,
  st.study_date AS series_date
FROM dbms.silver.series s
LEFT JOIN dbms.silver.study st
  ON s.study_uid = st.uid;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE TABLE IF NOT EXISTS dbms.gold.dim_image
SHALLOW CLONE dbms.silver.image;

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
2550331,1,null,0,0,0,0


In [0]:
%sql
CREATE TABLE dbms.gold.fact_patient (
  patient_id STRING NOT NULL,
  num_studies INT,
  last_study_date DATE
);

In [0]:
%sql
INSERT INTO dbms.gold.fact_patient
SELECT
  p.id AS patient_id,
  COUNT(st.uid) AS num_studies,
  MAX(st.study_date) AS last_study_date
FROM dbms.gold.dim_patient p
LEFT JOIN dbms.gold.dim_study st
  ON p.id = st.patient_id
GROUP BY p.id;

num_affected_rows,num_inserted_rows
575,575


In [0]:
%sql
CREATE TABLE dbms.gold.fact_study AS
SELECT
  st.uid AS study_uid,
  st.patient_id AS patient_id,
  COUNT(se.uid) AS num_series,
  ARRAY_DISTINCT(COLLECT_LIST(se.body_part)) AS body_part_list,
  ARRAY_DISTINCT(COLLECT_LIST(se.mpr)) AS mpr_list
FROM dbms.gold.dim_study st
LEFT JOIN dbms.gold.dim_series se
  ON st.uid = se.study_uid
GROUP BY st.uid, st.patient_id;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE TABLE dbms.gold.fact_series AS
SELECT
  se.uid AS series_uid,
  se.study_uid AS study_uid,
  st.patient_id AS patient_id,
  se.uid AS dim_series_uid,
  COUNT(img.sop_instance_uid) AS num_images
FROM dbms.gold.dim_series se
LEFT JOIN dbms.gold.dim_study st
  ON se.study_uid = st.uid
LEFT JOIN dbms.gold.dim_image img
  ON se.uid = img.series_uid
GROUP BY se.uid, se.study_uid, st.patient_id, se.uid;

num_affected_rows,num_inserted_rows


In [0]:
%sql
CREATE TABLE IF NOT EXISTS dbms.gold.fact_img AS
SELECT
  img.sop_instance_uid AS image_id,
  img.series_uid AS series_id,
  se.study_uid AS study_id,
  st.patient_id AS patient_id
FROM dbms.gold.dim_image img
LEFT JOIN dbms.gold.dim_series se
  ON img.series_uid = se.uid
LEFT JOIN dbms.gold.dim_study st
  ON se.study_uid = st.uid;

num_affected_rows,num_inserted_rows
